In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

1.8.0
True


In [3]:
datapath = 'data/'
train_df = pd.read_csv(datapath + 'train.csv')
train_df.head()
train_df.labels.value_counts()

0    197
2     76
1     55
Name: labels, dtype: int64

In [4]:
test_df = pd.read_csv(datapath + 'test.csv')
test_df.head()
test_df.labels.value_counts()

0    39
1    22
2    22
Name: labels, dtype: int64

In [ ]:
# train_df = pd.DataFrame({"text": train_df[1].replace(r"\n", " ", regex=True), "labels": train_df[0]})
# print(train_df.head())

In [ ]:
# test_df = pd.DataFrame({"text": test_df[1].replace(r"\n", " ", regex=True), "labels": test_df[0]})
# print(test_df.head())

In [5]:
# model_name = "bert-base-cased"
# model_type = "bert"

model_name = "distilroberta-base"
model_type = "roberta"


model_args = ClassificationArgs()
#model_args.no_cache = True
model_args.fp16 = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.output_dir = f"outputs/{model_type}"
model_args.best_model_dir = f"outputs/{model_type}/best_model"
model_args.evaluate_during_training = True
#model_args.lazy_loading = True
model_args.max_seq_length = 256
model_args.sliding_window = True
model_args.num_train_epochs = 12
model_args.train_batch_size = 64
model_args.eval_batch_size = 16

In [ ]:
#test_df.text.map(lambda x: len(x.split(' '))).max()

In [6]:
model = ClassificationModel(model_type, model_name, num_labels=3, weight=[2, 0.5, 1], args=model_args, use_cuda=True)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

In [7]:
model.train_model(train_df, eval_df=test_df)
#model.train_model(datapath + 'train.csv', eval_df=datapath + 'test.csv')

  0%|          | 0/328 [00:00<?, ?it/s]

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

Running Epoch 0 of 12:   0%|          | 0/15 [00:00<?, ?it/s]

C:\Miniconda3\envs\tesla_classifier\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Running Epoch 1 of 12:   0%|          | 0/15 [00:00<?, ?it/s]

...

Running Epoch 8 of 12:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 9 of 12:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 10 of 12:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 11 of 12:   0%|          | 0/15 [00:00<?, ?it/s]

(180,
 {'global_step': [15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180],
  'mcc': [0.0,
   0.11772040233700277,
   0.0,
   0.22606301504027465,
   0.16703176179554144,
   0.14931821064526266,
   0.26890726642491075,
   0.25243403836040756,
   0.2851596098255703,
   0.20843919616099224,
   0.25069865064023744,
   0.30229952676940114],
  'train_loss': [0.7609373331069946,
   0.5971140265464783,
   1.141303539276123,
   0.41461166739463806,
   0.31279030442237854,
   0.4328804612159729,
   0.0965457558631897,
   0.41755539178848267,
   0.07425758987665176,
   0.2568410336971283,
   0.18432246148586273,
   0.39721521735191345],
  'eval_loss': [0.9041745642820994,
   0.8221643328666687,
   0.8334376513957977,
   0.8501727163791657,
   0.8101803739865621,
   0.9671204666296641,
   0.8485099275906881,
   0.9419613639513652,
   1.0705350716908772,
   1.0874882479508718,
   1.085515554745992,
   1.1058086355527241]})

In [ ]:
preds, out = model.predict(["spin? tesla website hid normal steering wheel option to bizarre yoke. tesla's newest model announcements may have come with a side of clickbait. when the updated model s and x were revealed last week, their standout feature was a yoke-style steering wheel bereft of control stalks. the unusual design had previously been shown on several tesla prototypes, but never used in a production model. needless to say, it got plenty of attention in the automotive press. tesla's website says it provides \"the ultimate focus on driving: no stalks, no shifting. model s is the best car to drive, and the best car do be driven in.\". elon musk elaborated that the cars can \"guess\" what direction you want to go and that the transmission selector has been moved to the central touchscreen display for when you absolutely need to use it. the vehicles are also offered with tesla's optional full self-driving capability, which currently delivers a certain level of active driving assistance, but will eventually be capable of full autonomy, according to musk. the \"wheel\" has garnered interest from the national highway traffic safety administration, which said it was reaching out to tesla for more information. 250 mph tesla roadster delayed to 2022. however, it may not be the only one available. by modifying the source code of an image of the interior from the tesla consumer website, automotive outlet the drive discovered that an alternate one was hidden on the website that depicted it with a traditional round wheel. (tesla). it still doesn't have any stalks and the buttons on the spokes, which apparently control the turn signals, horn and lights, appear the same as those on the yoke. following the reporting on its discovery, the image was completely deleted by tesla, which has not commented on its reveal. currently, there is no mention of a steering wheel on the reservation page, which advertises that the first deliveries will begin in march, although musk said production is already underway and that shipments will start in february."])
print(preds)
print(out)

In [ ]:
train_df[:int(len(test_df)* 0.1)]